In [1]:
import bs4 as bs
import requests
import pandas as pd


In [2]:
#Llamada a la página web. Se traen las tablas, cada una contine datos de 1 libro
#Dos columnas, la de la izquierda con el nombre de los datos (categorías), la derecha con los datos
#Cada fila es una lista con dos elementos ([0] categorías; [1] datos). 28 filas por tabla

data = []


for i in range(1,400):
    url = 'http://catalogo.bibliotecas.gob.ar/pergamo/opac/cgi-bin/pgopac.cgi?VDOC=1.'+str(i) + '#Ejemplares'
    resp = requests.get(url)
    
    resp = resp.text
    soup = bs.BeautifulSoup(resp, 'html.parser')
    table = soup.find('table', class_='wsTableGrid', id='tbGeneral')

    if table is None:
        continue
    for row in (table.find_all('tr')):
        data.append(row.find_all('td'))     
      

In [3]:
len(data)

10005

In [4]:
#Creamos un diccionario para agrupar los datos según el nombre de cada uno. Y luego pasamos los datos agrupados a listas
#Resulta una lista (result) de 28 listas (cada una contiene los datos de las categorías de las tablas)
d = {}
for subl in data:
    try:
        if subl[0] in d:
            d[subl[0]].append(subl[1])
        else:
            d[subl[0]] = [subl[1]]
    except IndexError:
        continue
result = list(d.values())


In [5]:
len(d)

28

In [6]:
len(data)

10005

In [7]:
len(result)

28

In [8]:
#De las 399 iteraciones, quedan 299 puesto que en medio había listas vacías
len(result[4])

345

In [9]:
#Se crea una función para pasar los elementos de las listas de 'td' HTML a text
def pass_to_text(list_):
    text_list = []
    for tag in list_:
        text_list.append(tag.get_text())
     
    return(text_list)


In [10]:
#Se llama a la función para iterar sobre las listas con los datos que nos interesan
isbn = pass_to_text(result[0])
tipo_texto=pass_to_text(result[1])
titulo = pass_to_text(result[4])
autor = pass_to_text(result[7])
editorial = pass_to_text(result[10])
fecha_edicion = pass_to_text(result[11])
lugar_edicion = pass_to_text(result[13])
materia_0 = pass_to_text(result[26])
fecha_alta = pass_to_text(result[27])



In [11]:
# Limpiamos los strings de la lista materia_0 para que sólo quede el nombre de la materia
# sin perden elementos vacios

materia = []
for elem in materia_0:
    if elem:
        new_elem = ''.join(caracter
                           for caracter in elem
                           if caracter.isalpha() or
                           caracter.isspace() or
                           caracter == '-')
        materia.append(new_elem)
    else:
        materia.append(elem)


In [12]:
#Se crea un DataFrame con pandas
df = pd.DataFrame(list(zip(isbn,tipo_texto, titulo, autor, editorial, fecha_edicion, lugar_edicion, materia,fecha_alta)),
                  columns=['ISBN','Tipo_Texto', 'Título', 'Autor','Editorial','Fecha_Edición','Lugar_Edición','Materia','Fecha_Alta'])
df = df.reset_index(drop=True)
df


,ISBN,Tipo_Texto,Título,Autor,Editorial,Fecha_Edición,Lugar_Edición,Materia,Fecha_Alta
0,.,LITERATURA,Antología poética,"Marechal, Leopoldo (1900-1970); Andrés, Alfred...",Aguilar,1969,Buenos Aires,- LITERATURA ARGENTINA-POESIA,31/07/2001
1,950-515-107-1,LITERATURA,De mala muerte,"Abós, Álvaro (1941-...)",De la Flor,1986,Buenos Aires,- LITERATURA ARGENTINA-CUENTO,01/03/1996
2,84-261-0664-1,TEXTO NO LITERARIO,Confesiones,"Agustín, Santo (354-430); Esclassana, A. (trad.)",Juventud,1986,Barcelona,AGUSTIN FILOSOFIA-AGUSTIN SANTO -,01/03/1996
3,.,LITERATURA,El Capitán Veneno,"Alarcón, Pedro Antonio de (1833-1891)",Colihue,1985,Buenos Aires,- LITERATURA ESPAÑOLA-NOVELA,01/03/1996
4,.,LITERATURA,El sombrero de tres picos,"Alarcón, Pedro Antonio de (1833-1891)",Huemul,1971,Buenos Aires,- LITERATURA ESPAÑOLA-NOVELA,01/03/1996
...,...,...,...,...,...,...,...,...,...
340,950-13-2067-7,TEXTO NO LITERARIO,Nuevo castellano 1 : Lengua y literatura. Leng...,"Lacau, María Hortensia (1910-2006); Rosetti, M...",Kapelusz,1983,Buenos Aires,CASTELLANO-TEXTO SECUNDARIO LITERATURA -TEXTO...,12/03/1996
341,.,TEXTO NO LITERARIO,Nuevo castellano 2 : Lengua y literatura. Leng...,"Lacau, María Hortensia (1910-2006); Rosetti, M...",Kapelusz,1982,Buenos Aires,LENGUA -TEXTO SECUNDARIO LITERATURA -TEXTO SE...,12/03/1996
342,950-13-2069-3,TEXTO NO LITERARIO,Nuevo castellano 3 : Lengua y literatura. Leng...,"Lacau, María Hortensia (1910-2006); Rosetti, M...",Kapelusz,1982,Buenos Aires,LENGUA -TEXTO SECUNDARIO COMUNICACION-TEXTO S...,12/03/1996
343,950-46-0121-9,TEXTO NO LITERARIO,Ciencias biológicas 3,"Rubel, Diana Nora; Guahnon, Esther Liliana; La...",Santillana,1990,Buenos Aires,BIOLOGIA-TEXTO SECUNDARIO,12/03/1996


In [13]:
len(df)

345

In [14]:
#Se exportan los datos a archivos csv y xlsx

df.to_csv('library_catalog.csv', index=False)

df.to_excel("library_catalog.xlsx", index=False)


In [15]:
# Creamos diccionarios para unificar datos repetidos y agregar id sobre las columnas que nos interesan
autores = {autor: i for i, autor in enumerate(df['Autor'].unique())}
editoriales = {editorial: i for i,
               editorial in enumerate(df['Editorial'].unique())}
materias = {materia: i for i, materia in enumerate(df['Materia'].unique())}


In [16]:
# Copiamos los datos de la tabla original y mapeamos los datos de las columnas que nos interesan por su id
df2 = df.copy()
df2['Autor'] = df2['Autor'].map(autores)
df2['Editorial'] = df2['Editorial'].map(editoriales)
df2['Materia'] = df2['Materia'].map(materias)
df2


,ISBN,Tipo_Texto,Título,Autor,Editorial,Fecha_Edición,Lugar_Edición,Materia,Fecha_Alta
0,.,LITERATURA,Antología poética,0,0,1969,Buenos Aires,0,31/07/2001
1,950-515-107-1,LITERATURA,De mala muerte,1,1,1986,Buenos Aires,1,01/03/1996
2,84-261-0664-1,TEXTO NO LITERARIO,Confesiones,2,2,1986,Barcelona,2,01/03/1996
3,.,LITERATURA,El Capitán Veneno,3,3,1985,Buenos Aires,3,01/03/1996
4,.,LITERATURA,El sombrero de tres picos,3,4,1971,Buenos Aires,3,01/03/1996
...,...,...,...,...,...,...,...,...,...
340,950-13-2067-7,TEXTO NO LITERARIO,Nuevo castellano 1 : Lengua y literatura. Leng...,215,15,1983,Buenos Aires,189,12/03/1996
341,.,TEXTO NO LITERARIO,Nuevo castellano 2 : Lengua y literatura. Leng...,215,15,1982,Buenos Aires,190,12/03/1996
342,950-13-2069-3,TEXTO NO LITERARIO,Nuevo castellano 3 : Lengua y literatura. Leng...,215,15,1982,Buenos Aires,191,12/03/1996
343,950-46-0121-9,TEXTO NO LITERARIO,Ciencias biológicas 3,216,36,1990,Buenos Aires,192,12/03/1996


In [17]:
# Creamos nuevas tablas con id
autores_df = pd.DataFrame(list(autores.items()), columns=['Autor', 'ID'])
editoriales_df = pd.DataFrame(
    list(editoriales.items()), columns=['Editorial', 'ID'])
materias_df = pd.DataFrame(list(materias.items()), columns=['Materia', 'ID'])


In [18]:
# Se exportan los datos a archivos csv y xlsx

autores_df.to_csv('author_table.csv', index=False)

autores_df.to_excel("author_table.xlsx", index=False)


In [19]:
editoriales_df.to_csv('editorials_table.csv', index=False)

editoriales_df.to_excel("editorials_table.xlsx", index=False)


In [20]:
materias_df.to_csv('subjects_table.csv', index=False)

materias_df.to_excel("subjects_table.xlsx", index=False)
